useful link:
* [dataset](https://www.kaggle.com/jeanmidev/smart-meters-in-london?select=informations_households.csv)
* [data preview](https://medium.com/@boitemailjeanmid/smart-meters-in-london-part1-description-and-first-insights-jean-michel-d-db97af2de71b)

# Load Package

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from pandas import datetime
import os
import math
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(color_codes=True)

In [2]:
folder_path = './data/'
dataset_path = 'daily_dataset/'

# Acorn

## Data Cleaning

In [3]:
acorn = pd.read_csv(folder_path+'acorn_details.csv',encoding = "ISO-8859-1")
acorn.head()

,MAIN CATEGORIES,CATEGORIES,REFERENCE,ACORN-A,ACORN-B,ACORN-C,ACORN-D,ACORN-E,ACORN-F,ACORN-G,ACORN-H,ACORN-I,ACORN-J,ACORN-K,ACORN-L,ACORN-M,ACORN-N,ACORN-O,ACORN-P,ACORN-Q
0,POPULATION,Age,Age 0-4,77.0,83.0,72.0,100.0,120.0,77.0,97.0,97.0,63.0,119.0,67.0,114.0,113.0,89.0,123.0,138.0,133.0
1,POPULATION,Age,Age 5-17,117.0,109.0,87.0,69.0,94.0,95.0,102.0,106.0,67.0,95.0,64.0,108.0,116.0,86.0,89.0,136.0,106.0
2,POPULATION,Age,Age 18-24,64.0,73.0,67.0,107.0,100.0,71.0,83.0,89.0,62.0,104.0,459.0,97.0,96.0,86.0,117.0,109.0,110.0
3,POPULATION,Age,Age 25-34,52.0,63.0,62.0,197.0,151.0,66.0,90.0,88.0,63.0,132.0,145.0,109.0,96.0,90.0,140.0,120.0,120.0
4,POPULATION,Age,Age 35-49,102.0,105.0,91.0,124.0,118.0,93.0,102.0,103.0,76.0,111.0,67.0,99.0,98.0,90.0,102.0,103.0,100.0


In [8]:
# select only features we want to use
acorn_features = ['Age', 'Geography', 'Ethnicity', 'House Type', 'House Size',
                  'Children in household', 'Economic Activity', 'Car Ownership',
                  'Household Annual Income', 'Financial Situation',
                  'Internet Access: Frequency', 'TV on Demand',
                  'Environmental Groups', 'Action', 'Attitude']
acorn_clean = acorn[acorn.CATEGORIES.isin(acorn_features)].drop('MAIN CATEGORIES',1)

### Recoding features

In [9]:
# regroup and rename to recode features
def regroup(ways,data):
    for key,value in ways.items():
        data.loc[data.REFERENCE.isin(value),'REFERENCE'] = key
    return data

# def replace_string(ways,data,feature):
#     for way in ways:
#         data.loc[data.categories == feature,'reference'] = data.loc[acorn.categories == feature,'reference'].str.replace(way[0],way[1],regex=False)
#     return data

In [10]:
# for each feature dict([('new group 1',['old group 1','old group 2', 'old group 3']),
#                        ('new group 2',['old group 4','old group 5', 'old group 6'])])

categories = dict()
# 'Age'
age = dict([('child', ['Age 0-4', 'Age 5-17']),
            ('adult', ['Age 18-24', 'Age 25-34', 'Age 35-49','Age 50-64']),
            ('elderly', ['Aged 65-74', 'Aged 75 plus'])])
acorn_clean = regroup(age,acorn_clean)
categories['age'] = list(age.keys())
# 'Geography'
geography = dict([('england', ['England']),
                  ('northern_ireland', ['Northern Ireland']),
                  ('scotland', ['Scotland']),
                  ('wales',['Wales'])])
acorn_clean = regroup(geography,acorn_clean)
categories['geography'] = list(geography.keys())
# 'Ethnicity'
ethnicity = dict([('white', ['White']),
                  ('mixed', ['Mixed']),
                  ('asian', ['Asian']),
                  ('black',['Black']),
                  ('other_ethnicity',['Other ethnicity'])])
acorn_clean = regroup(ethnicity,acorn_clean)
categories['ethnicity'] = list(ethnicity.keys())
# House Type
house_type = dict([('detached_house', ['Bungalow', 'Detached house','Semi-detached house']),
                   ('flat', ['Flat or maisonette','Terraced house'])])
acorn_clean = regroup(house_type,acorn_clean)
# House Size
# house_size = dict([('bed_1', ['Number of Beds : 1']),
#                    ('bed_2', ['Number of Beds : 2']),
#                    ('bed_3', ['Number of Beds : 3']),
#                    ('bed_4', ['Number of Beds : 4']),
#                    ('bed_5', ['Number of Beds : 5 plus'])])
house_size = dict([('bed_1', ['Number of Beds : 1']),
                   ('bed_2', ['Number of Beds : 2']),
                   ('bed_3', ['Number of Beds : 3']),
                   ('bed_4', ['Number of Beds : 4','Number of Beds : 5 plus'])])
acorn_clean = regroup(house_size,acorn_clean)
categories['house_size'] = list(house_size.keys())
# Children in household
child = dict([('child_0', ['Children at home : 0']),
              ('child_1', ['Children at home : 1']),
              ('child_2', ['Children at home : 2']),
              ('child_3', ['Children at home : 3+'])])
acorn_clean = regroup(child,acorn_clean)
categories['child'] = list(child.keys())
# Economic Activity
employment = dict([('employed',['Employee Full-Time', 'Employee Part-Time', 'Self-employed','Other economic activity']),
                   ('unemployed', ['Retired', 'Unemployed and seeking work', 'Student','Looking after home or family'])])
acorn_clean = regroup(employment,acorn_clean)
categories['employment'] = list(employment.keys())
# Car Ownership
car = dict([('car_0', ['Number of Cars 0']),
           ('car_1', ['Number of Cars 1']),
           ('car_2', ['Number of Cars 2']),
           ('car_3', ['Number of Cars 3+'])])
acorn_clean = regroup(car,acorn_clean)
categories['car'] = list(car.keys())
# Household Annual Income
income = dict([('<20k', ['£0-£20,000']),
               ('20k-40k',['£20,000-£40,000']),
               ('40k-60k',['£40,000-£60,000']),
               ('60k-80k',['£60,000-£80,000']),
               ('80k-100k',['£80,000-£100,000']),
               ('>100k',['£100,000+']),
               ('average_level',['Average Household Income'])])
acorn_clean = regroup(income,acorn_clean)
categories['income'] = list(income.keys())
# Financial Situation
saving = dict([('saving', ['Saving', 'Saving a lot', 'Saving a little']),
               ('not_saving', ['Not saving', 'Just managing to make ends meet','Drawing on savings or running into debt'])])
acorn_clean = regroup(saving,acorn_clean)
categories['saving'] = list(saving.keys())
# Internet Access: Frequency
internet_freq = dict([('internet_high', ['Weekly']),
                      ('internet_med', ['Monthly']),
                      ('internet_low', ['Less than once a month','Never used the internet'])])
acorn_clean = regroup(internet_freq,acorn_clean)
categories['internet_freq'] = list(internet_freq.keys())
# TV on Demand
tv = dict([('tv', ['Watch on a TV set']),
           ('not_tv', ['Watch on a PC', 'Watch on a Mobile phone/Device'])])
acorn_clean = regroup(tv,acorn_clean)
categories['tv'] = list(tv.keys())
# Environmental Groups
environment_group = dict([('environment_group', ['Member of Environmental Group'])])
acorn_clean = regroup(environment_group,acorn_clean)
categories['environment_group'] = list(environment_group.keys())
# Action
action = dict([('water', ['Rarely keep the tap running while brushing teeth',
                        'Make an effort to cut down on water usage at home']),
               ('electricity', ['Rarely Leave TV or PC on standby for long periods of time at home',
                                'Rarely leave the heating on when out for a few hours',
                                'Rarely leave the mobile phone charger in the socket when not in use',
                                'Make an effort to cut down on the use of gas/electricity at home']),
               ('recycle', ['Re-use items like empty bottles, tubs, jars, envelopes or paper',
                            'Recycle items rather than throwing them away',
                            'Take own shopping bag when shopping',
                            'Decide not to buy something because it has too much packaging'])])
acorn_clean = regroup(action,acorn_clean)
categories['child'] = list(child.keys())
# Attitude
attitude = dict([('attitude', ['There is too much concern with the environment: Agree'])])
acorn_clean = regroup(attitude,acorn_clean)
categories['attitude'] = list(attitude.keys())

## convert index to percentage

In [11]:
# # percentage over category
# acorn_clean = acorn_clean.groupby(['CATEGORIES','REFERENCE']).sum(axis=1).reset_index()
# for feature in acorn_features:
#     values = acorn_clean[acorn_clean.CATEGORIES == feature].iloc[:,2:]
#     features = acorn_clean[acorn_clean.CATEGORIES == feature].iloc[:,:2]
#     if values.shape[0] == 1:
#         values = (values > 100).astype(int)
#     else:
#         values = values.divide(values.sum(axis=0),axis=1)
#     new_value = pd.concat([features,values],axis=1)
#     acorn_clean[acorn_clean.CATEGORIES == feature] = new_value
    
# acorn_clean = acorn_clean.drop('CATEGORIES',1).set_index('REFERENCE').T.reset_index()
# acorn_clean.columns = acorn_clean.columns.str.lower().str.replace(' ','_')
# acorn_clean.rename({'index':'Acorn'},axis=1,inplace=True)
# acorn_clean

In [12]:
# percentage over acorn groups
acorn_clean_boolean = acorn_clean.groupby(['CATEGORIES','REFERENCE']).sum(axis=1).reset_index()
index = acorn_clean_boolean.REFERENCE.isin(['attitude','environment_group']).values

sums = acorn_clean_boolean.iloc[~index,2:].sum(axis=1)
acorn_clean_boolean.iloc[~index,2:] = acorn_clean_boolean.iloc[~index,2:].divide(sums,axis=0)
acorn_clean_boolean.iloc[index,2:] = (acorn_clean_boolean.iloc[index,2:] > 100).astype(int)
acorn_clean_boolean = acorn_clean_boolean.drop('CATEGORIES',1).set_index('REFERENCE').T.reset_index()
acorn_clean_boolean.rename({'index':'Acorn'},axis=1,inplace=True)
acorn_clean_boolean

REFERENCE,Acorn,electricity,recycle,water,adult,child,elderly,attitude,car_0,car_1,...,60k-80k,80k-100k,<20k,>100k,average_level,internet_high,internet_low,internet_med,not_tv,tv
0,ACORN-A,0.058171,0.064323,0.063031,0.048746,0.058293,0.071493,0.0,0.011593,0.039859,...,0.117647,0.174914,0.011150,0.275769,0.113306,0.064831,0.040198,0.043906,0.060944,0.060048
1,ACORN-B,0.059651,0.063011,0.064790,0.051596,0.057692,0.067270,0.0,0.013105,0.028722,...,0.111176,0.142117,0.017606,0.179169,0.094712,0.064831,0.031960,0.035387,0.058083,0.065398
2,ACORN-C,0.061723,0.062427,0.063618,0.048461,0.047776,0.093213,0.0,0.012097,0.050996,...,0.078824,0.079402,0.038146,0.070696,0.067984,0.059667,0.049423,0.048493,0.050072,0.055886
3,ACORN-D,0.055062,0.062865,0.059513,0.071266,0.050781,0.031373,0.0,0.104839,0.055686,...,0.100000,0.120253,0.025235,0.144091,0.085997,0.067699,0.018451,0.022280,0.077825,0.063020
4,ACORN-E,0.058022,0.058926,0.061566,0.064282,0.064303,0.035596,0.0,0.059476,0.058617,...,0.089412,0.092060,0.032277,0.082029,0.073213,0.065404,0.025041,0.036697,0.076109,0.067182
5,ACORN-F,0.062463,0.067095,0.063911,0.050741,0.051683,0.078733,0.0,0.024194,0.046307,...,0.066471,0.057537,0.045775,0.039935,0.059849,0.059094,0.049753,0.053080,0.045780,0.052319
6,ACORN-G,0.060539,0.060823,0.061272,0.054732,0.059796,0.062142,1.0,0.015625,0.047479,...,0.083529,0.080552,0.035211,0.064220,0.068565,0.061388,0.042175,0.060288,0.056080,0.057669
7,ACORN-H,0.060983,0.059802,0.059513,0.055160,0.060998,0.060332,0.0,0.023185,0.055686,...,0.071176,0.062140,0.042840,0.043173,0.061592,0.059667,0.051071,0.062254,0.054649,0.067776
8,ACORN-I,0.065275,0.060968,0.063031,0.044612,0.039062,0.121267,1.0,0.023690,0.075615,...,0.034118,0.024166,0.076878,0.012952,0.043579,0.050488,0.093904,0.060288,0.034907,0.051130
9,ACORN-J,0.059651,0.059947,0.061566,0.062286,0.064303,0.041629,0.0,0.024194,0.060961,...,0.073529,0.063291,0.041080,0.042094,0.062173,0.062536,0.035255,0.057667,0.065236,0.065398


In [13]:
# percentage over acorn groups
acorn_clean_percentage = acorn_clean.groupby(['CATEGORIES','REFERENCE']).sum(axis=1).reset_index()
sums = acorn_clean_percentage.iloc[:,2:].sum(axis=1)
acorn_clean_percentage.iloc[:,2:] = acorn_clean_percentage.iloc[:,2:].divide(sums,axis=0)
acorn_clean_percentage = acorn_clean_percentage.drop('CATEGORIES',1).set_index('REFERENCE').T.reset_index()
acorn_clean_percentage.columns = acorn_clean_percentage.columns.str.lower().str.replace(' ','_')
acorn_clean_percentage.rename({'index':'Acorn'},axis=1,inplace=True)
acorn_clean_percentage

REFERENCE,Acorn,electricity,recycle,water,adult,child,elderly,attitude,car_0,car_1,...,60k-80k,80k-100k,<20k,>100k,average_level,internet_high,internet_low,internet_med,not_tv,tv
0,ACORN-A,0.058171,0.064323,0.063031,0.048746,0.058293,0.071493,0.055359,0.011593,0.039859,...,0.117647,0.174914,0.011150,0.275769,0.113306,0.064831,0.040198,0.043906,0.060944,0.060048
1,ACORN-B,0.059651,0.063011,0.064790,0.051596,0.057692,0.067270,0.054181,0.013105,0.028722,...,0.111176,0.142117,0.017606,0.179169,0.094712,0.064831,0.031960,0.035387,0.058083,0.065398
2,ACORN-C,0.061723,0.062427,0.063618,0.048461,0.047776,0.093213,0.055359,0.012097,0.050996,...,0.078824,0.079402,0.038146,0.070696,0.067984,0.059667,0.049423,0.048493,0.050072,0.055886
3,ACORN-D,0.055062,0.062865,0.059513,0.071266,0.050781,0.031373,0.057126,0.104839,0.055686,...,0.100000,0.120253,0.025235,0.144091,0.085997,0.067699,0.018451,0.022280,0.077825,0.063020
4,ACORN-E,0.058022,0.058926,0.061566,0.064282,0.064303,0.035596,0.055948,0.059476,0.058617,...,0.089412,0.092060,0.032277,0.082029,0.073213,0.065404,0.025041,0.036697,0.076109,0.067182
5,ACORN-F,0.062463,0.067095,0.063911,0.050741,0.051683,0.078733,0.058893,0.024194,0.046307,...,0.066471,0.057537,0.045775,0.039935,0.059849,0.059094,0.049753,0.053080,0.045780,0.052319
6,ACORN-G,0.060539,0.060823,0.061272,0.054732,0.059796,0.062142,0.061249,0.015625,0.047479,...,0.083529,0.080552,0.035211,0.064220,0.068565,0.061388,0.042175,0.060288,0.056080,0.057669
7,ACORN-H,0.060983,0.059802,0.059513,0.055160,0.060998,0.060332,0.058893,0.023185,0.055686,...,0.071176,0.062140,0.042840,0.043173,0.061592,0.059667,0.051071,0.062254,0.054649,0.067776
8,ACORN-I,0.065275,0.060968,0.063031,0.044612,0.039062,0.121267,0.060071,0.023690,0.075615,...,0.034118,0.024166,0.076878,0.012952,0.043579,0.050488,0.093904,0.060288,0.034907,0.051130
9,ACORN-J,0.059651,0.059947,0.061566,0.062286,0.064303,0.041629,0.058893,0.024194,0.060961,...,0.073529,0.063291,0.041080,0.042094,0.062173,0.062536,0.035255,0.057667,0.065236,0.065398


In [14]:
# save it to file
acorn_clean_boolean.to_csv(folder_path+'Acorn_cleaned_boolean.csv',index=False)
acorn_clean_percentage.to_csv(folder_path+'Acorn_cleaned_percentage.csv',index=False)

In [15]:
categories

{'age': ['child', 'adult', 'elderly'],
 'geography': ['england', 'northern_ireland', 'scotland', 'wales'],
 'ethnicity': ['white', 'mixed', 'asian', 'black', 'other_ethnicity'],
 'house_size': ['bed_1', 'bed_2', 'bed_3', 'bed_4'],
 'child': ['child_0', 'child_1', 'child_2', 'child_3'],
 'employment': ['employed', 'unemployed'],
 'car': ['car_0', 'car_1', 'car_2', 'car_3'],
 'income': ['<20k',
  '20k-40k',
  '40k-60k',
  '60k-80k',
  '80k-100k',
  '>100k',
  'average_level'],
 'saving': ['saving', 'not_saving'],
 'internet_freq': ['internet_high', 'internet_med', 'internet_low'],
 'tv': ['tv', 'not_tv'],
 'environment_group': ['environment_group'],
 'attitude': ['attitude']}